### Предварительные сведения

Векторные нормы:

$||u||_{\infty} = \max_i|u_i|$

$||u||_1 = \sum_i |u_i|$

$||u||_2 = \left(\sum_i |u_i|^2 \right)^{\frac{1}{2}}$

Матричные нормы:

$||A||_{\infty} = \max_i \sum_j |a_{ij}|$

$||A||_1 = \max_j \sum_i |a_{ij}|$

$||A||_2 = \left(\max_i \lambda_i(A A^*) \right)^{\frac{1}{2}}$

Контрольный вопрос: какова будет вторая норма матрицы, если матрица самосопряженная?

Ваш ответ на контрольный вопрос: 

$$AA^*=A^2\Rightarrow \lambda_i(A^2) = \lambda_i^2(A)$$

Значит, вторая норма равна максимальному(по модулю) модулю собственного числа самосопряженной матрицы 

In [1]:
import numpy as np
import numpy.linalg as la

A = np.array([[1,2],[3,4]])
v = range(0,3)
Vander = np.vander(v)
print('norm_1 = ', la.norm(Vander, 1))
print('norm_2 = ', la.norm(Vander, 2))
print('norm_inf = ', la.norm(Vander, np.inf))
Vander

norm_1 =  5.0
norm_2 =  4.844958524498339
norm_inf =  7.0


array([[0, 0, 1],
       [1, 1, 1],
       [4, 2, 1]])

Обусловленность:
$$(A+\delta A)u = f + \delta f$$
$$\frac{||\delta u||}{||u||}\le \frac{\mu}{1-\mu\frac{||\delta A||}{||A||}} \left(\frac{||\delta f||}{||f||}+\frac{||\delta A||}{||A||}\right)$$

$\mu$ - число обусловленности матрицы A, $\mu(A) = ||A^{-1}||\cdot||A||$, $\mu \ge 1$.



## Пример проблемы использования метода Гаусса для решения СЛАУ

In [2]:
import numpy as np
import numpy.linalg as la

def gauss( A, b ):
    n = b.size
    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - c*A[k,k+1:n]
                b[i] = b[i] - c*b[k]
                
    # обратный ход
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+1:n],b[k+1:n]))/A[k,k];
    return b

#все числа в представлены как вещественные
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

In [5]:
print(A1)

[[ 1.e-16  1.e+00 -1.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 2.e+00 -1.e+00  0.e+00]]


In [6]:
print(A2)

[[ 2.e+00 -1.e+00  0.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 1.e-16  1.e+00 -1.e+00]]


In [12]:
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

print('u1 = ', gauss(A1, b1))
#print('u1 = ', la.solve(A1, b1))
print('u2 = ', gauss(A2, b2))#la.solve(A2, b2))

mu1 =  16.393731622284385
mu2 =  16.393731622284392
u1 =  [0.55511151 0.25       0.25      ]
u2 =  [1. 1. 1.]


## Часть 1. LU разложение

Задание: 

реализовать алгоритм решения предыдущей задачи с матрицей A2 с помощью LU-разложение В решении должна выводиться L, U и собственно решение системы. 

ВАЖНО: реализация метода LU должна быть получена путем небольшой модификации метода gauss!  При это саму реализацию можно разделить на два метода: один метод собственно находит LU разложение (можно сделать переделкой цикла для матрицы A метода gauss), второй метод - непосредственное решение системы с помощью прямого и обратного хода. Ни в каком виде нельзя ползоваться пакетными методами (в частности, la.solve)

### LU - разложение с помощью пакета sympy

Чтобы убедиться, что разложение получено верно, можно воспользоваться скриптом ниже

In [33]:
import sympy as sp
import numpy as np

A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);
A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

A = sp.Matrix(A2)
L, U, _ = A.LUdecomposition()
U

Matrix([
[2.0, -1.0,                0.0],
[  0,  1.5,               -1.0],
[  0,    0, -0.333333333333333]])

In [2]:
import numpy as np
import numpy.linalg as la

def LUdecomp( A, b ):
    n = b.size
    L = np.eye(n)  
    U = A
    #LU - decomposition
    for k in range(0,n-1):
        for i in range(k+1,n):
            L[i,k] = U[i,k]/U[k,k]
            U[i,k:n] = U[i,k:n] - L[i,k]*U[k,k:n]
    print('L = ',L)
    print('U = ',U)
    y=np.zeros(n)
    x=np.zeros(n)
    # прямой ход
    for k in range(n):
        y[k] = (b[k] - sum([L[k,i]*y[i] for i in range(k)]))/L[k,k]

    # обратный ход
    for k in range(n-1,-1,-1):
        x[k] = (y[k] - sum([U[k,i]*x[i] for i in range(k+1,n)]))/U[k,k]

    return x

#все числа в представлены как вещественные
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('LUdec u2 = ', LUdecomp(A2,b2))

L =  [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.00000000e-01  1.00000000e+00  0.00000000e+00]
 [ 5.00000000e-17  6.66666667e-01  1.00000000e+00]]
U =  [[ 2.         -1.          0.        ]
 [ 0.          1.5        -1.        ]
 [ 0.          0.         -0.33333333]]
LUdec u2 =  [1. 1. 1.]


## Часть 2. Нахождение обратной матрицы с помощью LU разложения

Задание:

Предложить алгоритм с использованием LU-разложения и найти обратную матрицу с точностью $\epsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью функции numpy.inv.

In [3]:
import numpy as np
import sympy as sp
import numpy.linalg as la

def LUinv(A):
    n = np.shape(A)[0]
    L = np.eye(n)  
    U = A
    #LU - decomposition
    for k in range(0,n-1):
        for i in range(k+1,n):
            L[i,k] = U[i,k]/U[k,k]
            U[i,k:n] = U[i,k:n] - L[i,k]*U[k,k:n]
    print('L = ',L)
    print('U = ',U)
    y=np.zeros(n)
    X=np.zeros((n,n))
    #решаем систему для каждого столбца X
    for t in range(n):
        b = np.zeros(n)
        b[t]=1
        # прямой ход
        for k in range(n):
            y[k] = (b[k] - sum([L[k,i]*y[i] for i in range(k)]))/L[k,k]
        # обратный ход
        for k in range(n-1,-1,-1):
            X[k,t] = (y[k] - sum([U[k,i]*X[i,t] for i in range(k+1,n)]))/U[k,k]

    return X

A = np.array([[1,1,1],[0,1,2],[7,1,4]])
A1 = la.inv(A)
A2 = LUinv(A)
print('Ainv_with_linalg', A1)
print('Ainv = ', A2)
print('abs_error=' ,la.norm(A1-A2, ord = 1))

L =  [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 7. -6.  1.]]
U =  [[1 1 1]
 [0 1 2]
 [0 0 9]]
Ainv_with_linalg [[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]]
Ainv =  [[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]]
abs_error= 3.608224830031759e-16
